In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, KNNBasic

In [3]:

dados = []
with open('yelp_academic_dataset_business.json', 'r', encoding='utf-8') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha)) 

# Converter a lista de dicionários em um DataFrame
df_business = pd.DataFrame(dados)


In [4]:
dados = []
with open('yelp_academic_dataset_checkin.json', 'r', encoding='utf-8') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha)) 

# Converter a lista de dicionários em um DataFrame
df_checkin = pd.DataFrame(dados)

In [5]:
dados = []
with open('yelp_academic_dataset_review.json', 'r', encoding='utf-8') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha)) 

# Converter a lista de dicionários em um DataFrame
df_review = pd.DataFrame(dados)

In [6]:
dados = []
with open('yelp_academic_dataset_tip.json', 'r', encoding='utf-8') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha)) 

# Converter a lista de dicionários em um DataFrame
df_tip = pd.DataFrame(dados)

In [7]:
dados = []
with open('yelp_academic_dataset_user.json', 'r', encoding='utf-8') as arquivo:
    for linha in arquivo:
        dados.append(json.loads(linha)) 

# Converter a lista de dicionários em um DataFrame
df_user = pd.DataFrame(dados)

In [16]:
#guardar tudo em csvs para ser mais facil

df_business.to_csv('yelp_academic_dataset_business.csv',index=False)
df_checkin.to_csv('yelp_academic_dataset_checkin.csv',index=False)
df_review.to_csv('yelp_academic_dataset_review.csv',index=False)
df_tip.to_csv('yelp_academic_dataset_tip.csv',index=False)
df_user.to_csv('yelp_academic_dataset_user.csv',index=False)

In [9]:
df_business['city'].nunique()
for a in df_business['city'].unique():
    print(a)

Santa Barbara
Affton
Tucson
Philadelphia
Green Lane
Ashland City
Brentwood
St. Petersburg
Nashville
Land O' Lakes
Tampa Bay
Indianapolis
Clearwater
Largo
New Orleans
Kenner
Edmonton
Reno
Newtown
White House
Boise
Paoli
Ardmore
Exton
Wilmington
Edwardsville
Sparks
Alton
Cherry Hill
Bala Cynwyd
Springfield
Belleville
Carmel
Tampa
Kennett Square
Plymouth Meeting
Harvey
West Chester
Meridian
Hudson
Fernley
Williamstown
Pinellas Park
Glenolden
Wesley Chapel
Fishers
Burlington
Troy
Camden
Plainfield
Bensalem
Maplewood
Saint Louis
Fairview Heights
Oro Valley
Treasure Island
Southampton
Chalfont
Willow Grove
Voorhees
Tarpon Springs
Blue Bell
Metairie
Woodbury
Brownsburg
Norristown
Land O Lakes
Greenwood
Saint Petersburg
Brookhaven
Haverford
Glenside
Moorestown
Madison
Ewing
Levittown
Gloucester Township
Aston
Riverview
Warrington
Wyncote
St. Louis
Havertown
Madeira Beach
Lutz
Langhorne
King of Prussia
Smithton
Jenkintown
New Port Richey
Avon
Normandy
Wayne
Franklin
Tuscon
Brandon
Berwyn
TAMPA


In [13]:
#para escolher a cidade com mais opções

df_business['city'].value_counts()

Philadelphia      14569
Tucson             9250
Tampa              9050
Indianapolis       7540
Nashville          6971
                  ...  
Gentilly              1
pennsauken            1
Hamiltion             1
Newtown square        1
Apollo beach          1
Name: city, Length: 1416, dtype: int64

In [14]:
df_business_phila=df_business[df_business['city']=='Philadelphia']
df_business_phila

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
28,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"{'Smoking': 'u'no'', 'NoiseLevel': 'u'average'...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ..."
31,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3.0,41,0,"{'RestaurantsReservations': 'False', 'Caters':...","Pizza, Restaurants, Salad, Soup","{'Tuesday': '17:0-21:30', 'Wednesday': '17:0-1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150320,wPAaAbFbq5wfLfW0gnYU7g,Intermix,1718 Walnut St,Philadelphia,PA,19103,39.950060,-75.170129,2.0,10,0,"{'BusinessParking': '{'garage': False, 'street...","Fashion, Shopping, Women's Clothing","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'..."
150326,JDM7kiCEmeBc_5W-z6eZZw,2601 Parkway Apartments,2601 Pennsylvania Ave,Philadelphia,PA,19130,39.968806,-75.180608,3.0,6,1,{'BusinessAcceptsCreditCards': 'True'},"Real Estate, Apartments, Home Services",None
150329,9U1Igcpe954LoWZRmNc-zg,Hand & Stone Massage And Facial Spa,"1100 S Columbus Blvd, Ste 24",Philadelphia,PA,19147,39.932756,-75.144504,3.0,32,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Day Spas, Beauty & Spas, Skin Care, Massage","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1",Philadelphia,PA,19103,39.945966,-75.169666,4.5,39,1,"{'ByAppointmentOnly': 'True', 'GoodForKids': '...","Health & Medical, Yoga, Shopping, Naturopathic...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


In [15]:
print(df_business.columns)
print(df_checkin.columns)
print(df_review.columns)
print(df_tip.columns)
print(df_user.columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
Index(['business_id', 'date'], dtype='object')
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')
Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')
Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')


In [17]:
#testar outra forma de caregar os ficheiros
businesses_df = pd.read_json('yelp_academic_dataset_business.json', lines=True)
businesses_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [18]:
ver = pd.read_csv('yelp_academic_dataset_business.csv')
print(len(businesses_df))
print(len(ver))

#confirmar que dá tudo o mesmo

150346
150346


In [19]:
#fiz assim se não ia demorar muito tempo

reviews_df = pd.read_csv('yelp_academic_dataset_review.csv')

In [20]:
print('Restaurants' in businesses_df['categories'].unique())

True


In [21]:
# 2. Filtrar apenas os restaurantes em Filadélfia
philly_restaurants = businesses_df[(businesses_df['city'] == 'Philadelphia') & (businesses_df['categories'].str.contains('Restaurants', na=False))].reset_index(drop=True)

ratings = reviews_df[reviews_df['business_id'].isin(philly_restaurants['business_id'])]
ratings=ratings[['user_id', 'business_id', 'stars']]



Aqui estamos a fazer a análise textual às categorias. Podemos perguntar à prof se devemos usar estas categorias como features categóricas simplesmente em vez de analisar o texto, e aplicar a análise textual às reviews.

In [67]:
tfidf = TfidfVectorizer(stop_words='english')
philly_restaurants['categories'] = philly_restaurants['categories'].fillna('')

# Converter categorias em vetores numéricos
matrix = tfidf.fit_transform(philly_restaurants['categories'])

# 4. Similaridade entre restaurantes com base nas categorias (similaridade do cosseno)
sim = cosine_similarity(matrix, matrix)

In [26]:
combined_similarity = sim

In [120]:
philly_restaurants

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,Philadelphia,PA,19106,39.953949,-75.143226,4.0,245,1,"{'RestaurantsReservations': 'True', 'Restauran...","Sushi Bars, Restaurants, Japanese","{'Tuesday': '13:30-22:0', 'Wednesday': '13:30-..."
2,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,Philadelphia,PA,19147,39.943223,-75.162568,4.5,205,1,"{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...","Korean, Restaurants","{'Monday': '11:30-20:30', 'Tuesday': '11:30-20..."
3,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4.0,65,0,"{'Smoking': 'u'no'', 'NoiseLevel': 'u'average'...","Cocktail Bars, Bars, Italian, Nightlife, Resta...","{'Monday': '16:0-0:0', 'Tuesday': '16:0-0:0', ..."
4,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3.0,41,0,"{'RestaurantsReservations': 'False', 'Caters':...","Pizza, Restaurants, Salad, Soup","{'Tuesday': '17:0-21:30', 'Wednesday': '17:0-1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,VZbkSeZtFynEascotq7ExA,Ali Baba Magic Food,37TH And Walnut,Philadelphia,PA,19104,39.953391,-75.196765,4.0,8,0,"{'GoodForKids': 'False', 'RestaurantsReservati...","Restaurants, Food Stands",None
5848,gPr1io7ks0Eo3FDsnDTYfg,Tata Cafe,7201 Germantown Ave,Philadelphia,PA,19119,40.060414,-75.191084,4.0,21,0,"{'RestaurantsGoodForGroups': 'True', 'Business...","Sandwiches, Restaurants, Italian","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
5849,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,Philadelphia,PA,19128,40.032483,-75.214430,3.0,55,1,"{'NoiseLevel': 'u'average'', 'RestaurantsTakeO...","Restaurants, Specialty Food, Food, Sandwiches,...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
5850,8n93L-ilMAsvwUatarykSg,Kitchen Gia,3716 Spruce St,Philadelphia,PA,19104,39.951018,-75.198240,3.0,22,0,"{'RestaurantsGoodForGroups': 'True', 'BikePark...","Coffee & Tea, Food, Sandwiches, American (Trad...","{'Monday': '9:0-19:30', 'Tuesday': '9:0-19:30'..."


In [37]:
# 7. Criar uma função para recomendar restaurantes com base na similaridade
def recommend_similar_restaurants(restaurant_id, philly_restaurants, similarity_matrix, n=5):
    # Obter o índice do restaurante
    idx = philly_restaurants.index[0] # ISTO VAI SEMPRE BUSCAR 0, NÃO? idx VAI SER SEMPRE 0
    
    # Obter similaridade do restaurante com todos os outros
    similars_indices = sim[idx].argsort()[::-1]  # Do mais similar para o menos similar
    
    # Excluir o próprio restaurante da recomendação
    similars_indices = similars_indices[similars_indices != idx]
    
    # Selecionar os n mais similares
    similars_restaurants = philly_restaurants.iloc[similars_indices[:n]]
    
    return similars_restaurants[['name', 'categories', 'stars']]

In [138]:
# acho que devemos usar algo assim
philly_restaurants['business_id'][philly_restaurants['business_id'] == 'MTSW4McQd7CbVtyjqoe9mw'].index

Int64Index([0], dtype='int64')

In [107]:
# Configurar o formato do Reader para o Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings, reader)

In [110]:
# Treinamento de modelo KNN (Item-based CF)
sim_options = {
    'name': 'cosine',
    'user_based': False  # Item-based CF
}

algo = KNNBasic(sim_options=sim_options)
trainset = data.build_full_trainset()
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [111]:
# 9. Função para recomendar com base nos restaurantes que o usuário já avaliou bem
def recommend_for_user(user_id, philly_restaurants, algo, similarity_matrix, n=5):
    # Obter os restaurantes avaliados pelo usuário
    user_reviews = reviews_df[reviews_df['user_id'] == user_id]
    highly_rated = user_reviews[user_reviews['stars'] >= 4]['business_id']
    # Para cada restaurante que o usuário gostou, recomendar restaurantes similares
    recommendations = pd.DataFrame()
    
    for restaurant_id in highly_rated:
        similar_restaurants = recommend_similar_restaurants(restaurant_id, philly_restaurants, similarity_matrix, n)
        recommendations = pd.concat([recommendations, similar_restaurants])
    
    # Remover duplicatas e ordenar por popularidade (opcional: você pode melhorar o critério de ordenação)
    recommendations = recommendations.drop_duplicates(subset='name').sort_values(by='stars', ascending=False)
    
    return recommendations.head(n)

In [139]:
ratings['user_id'].value_counts()

ET8n-r7glWYqZhuR6GcdNw    929
_BcWyKQL16ndpBdggh2kNA    787
bJ5FtCtZX3ZZacz2_2PJjA    663
vFd8aBLg1kFcd0kCkoi-xw    529
8EMU7d4pCkdqUnvlIW40CA    469
                         ... 
CggzCzm5TIezdLtK-p85aQ      1
U9UKS303tt_F8hA2ONfOFQ      1
71hPJYl24X1xEKRIIroDxw      1
nPs4yF-EfPgGzwHmBk07Qg      1
384sxlFjXXnSdKa1l0IxyQ      1
Name: user_id, Length: 209513, dtype: int64

In [140]:
# 10. Testar recomendação para um usuário específico
user_id = '8EMU7d4pCkdqUnvlIW40CA'  
recommendations = recommend_for_user(user_id, philly_restaurants, algo, combined_similarity, n=5)
print(f"Recomendações para o usuário {user_id}:")
print(recommendations)


Recomendações para o usuário 8EMU7d4pCkdqUnvlIW40CA:
                    name                                         categories  \
3075              T cafe  Bakeries, Desserts, Bubble Tea, Coffee & Tea, ...   
5065          Vegan Tree     Bakeries, Bubble Tea, Food, Vegan, Restaurants   
4951         Tiger Sugar  Desserts, Coffee & Tea, Food, Restaurants, Bub...   
5008  Ponte Coffee House          Restaurants, Food, Bakeries, Coffee & Tea   
3543   Zhong Gang Bakery          Food, Bakeries, Coffee & Tea, Restaurants   

      stars  
3075    4.5  
5065    4.0  
4951    4.0  
5008    3.5  
3543    3.5  
